Import Libraries

In [1]:
import requests
import pandas as pd
from datetime import datetime
import mysql.connector

Fetch Data from API

In [2]:
api_url = "https://fakestoreapi.com/products"

In [3]:
response = requests.get(api_url)

In [4]:
if response.status_code == 200:
    data = response.json()
    print("Data fetched successfully!")
else:
    print("Failed to fetch data:", response.status_code)

Data fetched successfully!


Data Analysing

In [5]:
df = pd.DataFrame(data)

In [6]:
print(df.head())

   id                                              title   price  \
0   1  Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...  109.95   
1   2             Mens Casual Premium Slim Fit T-Shirts    22.30   
2   3                                 Mens Cotton Jacket   55.99   
3   4                               Mens Casual Slim Fit   15.99   
4   5  John Hardy Women's Legends Naga Gold & Silver ...  695.00   

                                         description        category  \
0  Your perfect pack for everyday use and walks i...  men's clothing   
1  Slim-fitting style, contrast raglan long sleev...  men's clothing   
2  great outerwear jackets for Spring/Autumn/Wint...  men's clothing   
3  The color could be slightly different between ...  men's clothing   
4  From our Legends Collection, the Naga was insp...        jewelery   

                                               image  \
0  https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...   
1  https://fakestoreapi.com/img/71-3HjGNDUL._A

In [10]:
print(df.tail())

    id                                              title  price  \
15  16  Lock and Love Women's Removable Hooded Faux Le...  29.95   
16  17  Rain Jacket Women Windbreaker Striped Climbing...  39.99   
17  18        MBJ Women's Solid Short Sleeve Boat Neck V    9.85   
18  19                 Opna Women's Short Sleeve Moisture   7.95   
19  20         DANVOUY Womens T Shirt Casual Cotton Short  12.99   

                                          description          category  \
15  100% POLYURETHANE(shell) 100% POLYESTER(lining...  women's clothing   
16  Lightweight perfet for trip or casual wear---L...  women's clothing   
17  95% RAYON 5% SPANDEX, Made in USA or Imported,...  women's clothing   
18  100% Polyester, Machine wash, 100% cationic po...  women's clothing   
19  95%Cotton,5%Spandex, Features: Casual, Short S...  women's clothing   

                                                image  \
15  https://fakestoreapi.com/img/81XH0e8fefL._AC_U...   
16  https://fakestoreapi.c

In [11]:
print(df.columns)

Index(['id', 'title', 'price', 'description', 'category', 'image', 'rating'], dtype='object')


In [12]:
print(df.info)

<bound method DataFrame.info of     id                                              title   price  \
0    1  Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...  109.95   
1    2             Mens Casual Premium Slim Fit T-Shirts    22.30   
2    3                                 Mens Cotton Jacket   55.99   
3    4                               Mens Casual Slim Fit   15.99   
4    5  John Hardy Women's Legends Naga Gold & Silver ...  695.00   
5    6                       Solid Gold Petite Micropave   168.00   
6    7                         White Gold Plated Princess    9.99   
7    8  Pierced Owl Rose Gold Plated Stainless Steel D...   10.99   
8    9  WD 2TB Elements Portable External Hard Drive -...   64.00   
9   10  SanDisk SSD PLUS 1TB Internal SSD - SATA III 6...  109.00   
10  11  Silicon Power 256GB SSD 3D NAND A55 SLC Cache ...  109.00   
11  12  WD 4TB Gaming Drive Works with Playstation 4 P...  114.00   
12  13  Acer SB220Q bi 21.5 inches Full HD (1920 x 108...  599.00   
13

In [13]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           20 non-null     int64  
 1   title        20 non-null     object 
 2   price        20 non-null     float64
 3   description  20 non-null     object 
 4   category     20 non-null     object 
 5   image        20 non-null     object 
 6   rating       20 non-null     object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.2+ KB
None
